In [1]:
 import os
 import numpy as np
 import pandas as pd
 import tensorflow as tf
 
 import matplotlib.pyplot as plt
 
 from freeman.plt_setting import plt_settings
 from freeman.tensor import MakeWindow

2023-09-05 21:57:25.147211: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
plt_settings()

In [3]:
_PATH_BASE = os.path.join(os.getcwd(), 'data')
_PATH_CLIMATE = os.path.join(_PATH_BASE, 'climate_pp.csv')

try:
    df_climate = pd.read_csv(_PATH_CLIMATE)
except Exception as e:
    print('파일이 존재하지 않습니다.\n'
          '52-ts-climate-pp.ipynb 실행 후 작업하시기 바랍니다.')
    print(e)

In [4]:
WINDOW_SIZE = 24
BATCH_SIZE = 32
TARGET_COL = 'T (degC)'

In [5]:
win_data = MakeWindow(
    df_climate, WINDOW_SIZE, BATCH_SIZE, label_col=TARGET_COL
)

2023-09-05 21:57:26.864246: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 21:57:26.868772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 21:57:26.868925: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Batch dataset shape: features(32, 24, 19), label(32,)
Last batch dataset shape: features(25, 24, 19), label(25,)
Batch dataset size: 2191
Total dataset row size: 70105 = 32 * (2191 - 1) + 25


In [6]:
win_data.slices()

Train Data:
Batch dataset shape: features(32, 24, 19), label(32,)
Last batch dataset shape: features(32, 24, 19), label(32,)
Batch dataset size: 1314
Total dataset row size: 42048 = 32 * (1314 - 1) + 32
Valid Data:
Batch dataset shape: features(32, 24, 19), label(32,)
Last batch dataset shape: features(32, 24, 19), label(32,)
Batch dataset size: 438
Total dataset row size: 14016 = 32 * (438 - 1) + 32
Test Data:
Batch dataset shape: features(32, 24, 19), label(32,)
Last batch dataset shape: features(25, 24, 19), label(25,)
Batch dataset size: 439
Total dataset row size: 14041 = 32 * (439 - 1) + 25


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, activation='tanh', input_shape=win_data.input_shape),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()]
)

In [8]:
model.fit(       # return history
    win_data.train, 
    epochs=10, 
    validation_data=win_data.valid,
)

Epoch 1/10


2023-09-05 21:57:40.521839: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-09-05 21:57:40.784542: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd1d01d2840 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-05 21:57:40.784565: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2023-09-05 21:57:40.840516: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


      1/Unknown - 3s 3s/step - loss: 25.6008 - mean_absolute_error: 4.8079

2023-09-05 21:57:41.036789: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-09-05 21:57:41.139426: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


   1296/Unknown - 7s 3ms/step - loss: 114.6908 - mean_absolute_error: 8.7901

2023-09-05 21:57:44.926385: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10558366348436227318
2023-09-05 21:57:44.926425: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 13163502417316224400


1314/1314 [==============================] - 10s 5ms/step - loss: 113.5534 - mean_absolute_error: 8.7355 - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/10
  56/1314 [>.............................] - ETA: 3s - loss: 59.7211 - mean_absolute_error: 5.9970

2023-09-05 21:57:47.988382: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10558366348436227318
2023-09-05 21:57:47.988418: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 13163502417316224400


1314/1314 [==============================] - 6s 5ms/step - loss: 84.2717 - mean_absolute_error: 7.5113 - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/10
1314/1314 [==============================] - 6s 5ms/step - loss: 78.9606 - mean_absolute_error: 7.2646 - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/10
1314/1314 [==============================] - 6s 5ms/step - loss: 78.3298 - mean_absolute_error: 7.2261 - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/10
1314/1314 [==============================] - 6s 5ms/step - loss: 78.3344 - mean_absolute_error: 7.2211 - val_loss: nan - val_mean_absolute_error: nan
Epoch 6/10
1314/1314 [==============================] - 6s 5ms/step - loss: 78.5920 - mean_absolute_error: 7.2343 - val_loss: nan - val_mean_absolute_error: nan
Epoch 7/10
1314/1314 [==============================] - 6s 5ms/step - loss: 78.4637 - mean_absolute_error: 7.2270 - val_loss: nan - val_mean_absolute_error: nan
Epoch 8/10
1314/1314 [=======================

In [9]:
model.evaluate(win_data.test)

439/439 [==============================] - 3s 2ms/step - loss: nan - mean_absolute_error: nan


2023-09-05 21:59:50.153822: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10558366348436227318


[nan, nan]